server only

In [ ]:
import flwr as fl
import torch
import torch.nn as nn
import numpy as np
import logging
import os
import sys
import json
from typing import List, Tuple, Dict, Optional, Union
from flwr.common import Parameters, FitRes, EvaluateRes, Scalar
from flwr.common import parameters_to_ndarrays, ndarrays_to_parameters
from collections import OrderedDict

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("server.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("FraminghamServer")

# Server configuration
SERVER_CONFIG = {
    "num_rounds": 10,
    "min_available_clients": 2,
    "min_fit_clients": 2,
    "min_evaluate_clients": 2,
    "input_size": 15,  # This should match the number of features in your dataset
    "save_dir": "model_checkpoints",
    "aggregation": "fedprox",  # Options: "fedavg", "fedprox"
    "proximal_mu": 0.01  # FedProx hyperparameter (should match clients)
}

# Same model definition as the client
class HeartDiseaseModel(nn.Module):
    def __init__(self, input_size):
        super(HeartDiseaseModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

# Aggregation function for FedProx (server-side)
def federated_averaging(results):
    """Compute weighted average of model updates based on number of examples."""
    # Extract weights and num_examples
    weights_results = [(parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples) 
                      for _, fit_res in results]
    
    # Get the total number of examples
    total_examples = sum([num_examples for _, num_examples in weights_results])
    
    # Log the weights distribution
    logger.info(f"Aggregating from {len(weights_results)} clients with a total of {total_examples} examples")
    
    if total_examples == 0:
        # Handle edge case of no examples
        return None
    
    # Compute weighted average
    weighted_weights = []
    for weights, num_examples in weights_results:
        # Skip clients that had no examples
        if num_examples == 0:
            continue
            
        # Calculate weight for this client based on number of examples
        weight = num_examples / total_examples
        
        # Apply weighted contribution to each layer
        if not weighted_weights:  # First client, initialize weighted_weights
            weighted_weights = [layer * weight for layer in weights]
        else:  # Add to the existing weighted sum
            for i, layer in enumerate(weights):
                weighted_weights[i] += layer * weight
    
    return weighted_weights

# Custom strategy for FedProx
class FedProxStrategy(fl.server.strategy.FedAvg):
    """Custom strategy for Framingham study with FedProx support"""
    
    def __init__(self, initial_parameters, save_dir=None):
        super().__init__(
            fraction_fit=1.0,  # Use all available clients for training
            fraction_evaluate=1.0,  # Use all available clients for evaluation
            min_fit_clients=SERVER_CONFIG["min_fit_clients"],
            min_evaluate_clients=SERVER_CONFIG["min_evaluate_clients"],
            min_available_clients=SERVER_CONFIG["min_available_clients"],
        )
        self.initial_parameters = initial_parameters
        self.save_dir = save_dir or "model_checkpoints"
        self.metrics_history = []
        
        # Create directory for saving models
        os.makedirs(self.save_dir, exist_ok=True)
    
    def initialize_parameters(self, client_manager):
        """Initialize model parameters"""
        return self.initial_parameters
    
    def aggregate_fit(self, server_round, results, failures):
        """Aggregate model updates from clients"""
        if not results:
            logger.warning(f"Round {server_round}: No results to aggregate!")
            return None, {}
        
        # Log aggregation info
        logger.info(f"Round {server_round}: Aggregating updates from {len(results)} clients")
        
        # Extract client metrics for logging
        client_metrics = {}
        for client_proxy, fit_res in results:
            client_id = client_proxy.cid
            metrics = fit_res.metrics if fit_res.metrics else {}
            client_metrics[client_id] = metrics
            
            # Log details for each client
            logger.info(f"Client {client_id} - Examples: {fit_res.num_examples}")
            for key, value in metrics.items():
                logger.info(f"Client {client_id} - {key}: {value}")
        
        # Perform aggregation - same as FedAvg for the server side
        # (FedProx's main difference is in the client's loss function)
        if len(results) > 0:
            # Aggregate parameters using federated averaging
            aggregated_ndarrays = federated_averaging(results)
            
            if aggregated_ndarrays is not None:
                aggregated_parameters = ndarrays_to_parameters(aggregated_ndarrays)
                
                # Add round number to metrics
                metrics = {"round": server_round}
                
                # Calculate average metrics across all clients 
                if len(client_metrics) > 0:
                    avg_accuracy = np.mean([
                        m.get("accuracy", 0.0) for m in client_metrics.values()
                    ])
                    avg_loss = np.mean([
                        m.get("loss", 0.0) for m in client_metrics.values()
                    ])
                    metrics["accuracy"] = float(avg_accuracy)
                    metrics["loss"] = float(avg_loss)
                
                # Save the aggregated model
                self._save_model(server_round, aggregated_ndarrays)
                
                # Store metrics for history
                self.metrics_history.append({
                    "round": server_round,
                    "client_metrics": client_metrics,
                    "aggregated_metrics": metrics,
                })
                self._save_metrics()
                
                return aggregated_parameters, metrics
        
        # Fallback if aggregation fails
        return None, {}
    
    def aggregate_evaluate(self, server_round, results, failures):
        """Aggregate evaluation results from clients"""
        if not results:
            return None, {}
        
        # Log evaluation info
        logger.info(f"Round {server_round}: Aggregating evaluation from {len(results)} clients")
        
        # Extract and aggregate metrics across clients
        accuracies = [r.metrics.get("accuracy", 0.0) for _, r in results]
        losses = [r.loss for _, r in results]
        
        # Compute averages
        avg_accuracy = sum(accuracies) / len(accuracies) if accuracies else 0.0
        avg_loss = sum(losses) / len(losses) if losses else 0.0
        
        # Final metrics
        metrics = {
            "accuracy": float(avg_accuracy),
            "loss": float(avg_loss),
        }
        
        logger.info(f"Round {server_round} evaluation - Accuracy: {avg_accuracy:.4f}, Loss: {avg_loss:.4f}")
        
        return avg_loss, metrics
    
    def _save_model(self, round_num, parameters):
        """Save model checkpoint"""
        try:
            # Create directory if it doesn't exist
            os.makedirs(self.save_dir, exist_ok=True)
            
            # Convert parameters to PyTorch model
            model = HeartDiseaseModel(input_size=SERVER_CONFIG["input_size"])
            
            # Fix the parameter shapes before saving
            # This ensures parameters have consistent format and avoids inhomogeneous array errors
            state_dict = OrderedDict()
            model_state_dict = model.state_dict()
            
            for i, (name, _) in enumerate(model_state_dict.items()):
                if i < len(parameters):
                    state_dict[name] = torch.tensor(parameters[i])
            
            model.load_state_dict(state_dict)
            
            # Save PyTorch model
            model_path = os.path.join(self.save_dir, f"model_round_{round_num}.pt")
            torch.save(model.state_dict(), model_path)
            
            logger.info(f"Saved model checkpoint for round {round_num}")
            
            # Also save raw parameters as NumPy arrays 
            params_path = os.path.join(self.save_dir, f"params_round_{round_num}.npz")
            np.savez(params_path, *parameters)
            
        except Exception as e:
            logger.error(f"Failed to save model: {str(e)}", exc_info=True)
    
    def _save_metrics(self):
        """Save training metrics history"""
        try:
            metrics_path = os.path.join(self.save_dir, "metrics_history.json")
            with open(metrics_path, "w") as f:
                json.dump(self.metrics_history, f, indent=2)
            logger.info(f"Saved metrics history to {metrics_path}")
        except Exception as e:
            logger.error(f"Failed to save metrics: {str(e)}")

def start_server(
    num_rounds=None, 
    min_clients=None, 
    input_size=None, 
    save_dir=None,
    aggregation=None,
    proximal_mu=None
):
    """Start the federated learning server"""
    # Update configuration if values provided
    if num_rounds is not None:
        SERVER_CONFIG["num_rounds"] = num_rounds
    if min_clients is not None:
        SERVER_CONFIG["min_available_clients"] = min_clients
        SERVER_CONFIG["min_fit_clients"] = min_clients
        SERVER_CONFIG["min_evaluate_clients"] = min_clients
    if input_size is not None:
        SERVER_CONFIG["input_size"] = input_size
    if save_dir is not None:
        SERVER_CONFIG["save_dir"] = save_dir
    if aggregation is not None:
        SERVER_CONFIG["aggregation"] = aggregation
    if proximal_mu is not None:
        SERVER_CONFIG["proximal_mu"] = proximal_mu
    
    # Create directory for saving models
    os.makedirs(SERVER_CONFIG["save_dir"], exist_ok=True)
    
    # Initialize model 
    model = HeartDiseaseModel(input_size=SERVER_CONFIG["input_size"])
    
    # Get model parameters as numpy arrays
    initial_params = [val.detach().cpu().numpy() for val in model.parameters()]
    
    # Convert to Flower parameters format
    initial_parameters = ndarrays_to_parameters(initial_params)
    
    # Create strategy (using the same strategy regardless of FedAvg or FedProx since the
    # main FedProx logic is in the client loss function)
    strategy = FedProxStrategy(
        initial_parameters=initial_parameters,
        save_dir=SERVER_CONFIG["save_dir"]
    )
    
    # Print configuration
    print(f"\n===== Framingham Heart Study FL Server ({SERVER_CONFIG['aggregation']}) =====")
    print(f"Rounds:          {SERVER_CONFIG['num_rounds']}")
    print(f"Minimum clients: {SERVER_CONFIG['min_available_clients']}")
    print(f"Input size:      {SERVER_CONFIG['input_size']}")
    print(f"Save directory:  {SERVER_CONFIG['save_dir']}")
    if SERVER_CONFIG['aggregation'] == 'fedprox':
        print(f"Proximal mu:     {SERVER_CONFIG['proximal_mu']}")
    print("=============================================")
    print("\nWaiting for clients to connect...\n")
    
    # Start server
    server_config = fl.server.ServerConfig(num_rounds=SERVER_CONFIG["num_rounds"])
    return fl.server.start_server(
        server_address="0.0.0.0:8080",
        strategy=strategy,
        config=server_config
    )

if __name__ == "__main__":
    import argparse
    
    # Create argument parser
    parser = argparse.ArgumentParser(description="Framingham Heart Study FL Server")
    parser.add_argument("--rounds", type=int, default=10, help="Number of federated learning rounds")
    parser.add_argument("--min_clients", type=int, default=2, help="Minimum number of clients")
    parser.add_argument("--input_size", type=int, default=15, help="Number of input features")
    parser.add_argument("--save_dir", type=str, default="model_checkpoints", help="Directory to save models")
    parser.add_argument("--aggregation", type=str, default="fedprox", choices=["fedavg", "fedprox"], 
                      help="Aggregation method")
    parser.add_argument("--mu", type=float, default=0.01, help="FedProx proximal term strength")
    
    # Handle case when running in Jupyter
    if 'ipykernel' in sys.modules:
        print("Running in Jupyter/IPython environment")
        start_server()
    else:
        # Parse arguments and start server
        args = parser.parse_args()
        
        try:
            start_server(
                num_rounds=args.rounds,
                min_clients=args.min_clients, 
                input_size=args.input_size,
                save_dir=args.save_dir,
                aggregation=args.aggregation,
                proximal_mu=args.mu
            )
        except Exception as e:
            logger.error(f"Server failed: {str(e)}", exc_info=True)

	Instead, use the `flower-superlink` CLI command to start a SuperLink as shown below:

		$ flower-superlink --insecure

	To view usage and all available options, run:

		$ flower-superlink --help

	Using `start_server()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
2025-05-24 11:39:44,644 - flwr - WARNING - DEPRECATED FEATURE: flwr.server.start_server() is deprecated.
	Instead, use the `flower-superlink` CLI command to start a SuperLink as shown below:

		$ flower-superlink --insecure

	To view usage and all available options, run:

		$ flower-superlink --help

	Using `start_server()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower server, config: num_rounds=10, no round_timeout
2025-05-24 11:39:44,645 - flwr - INFO - Starting Flower server, config: num_rounds=10, no round_timeout

Running in Jupyter/IPython environment

===== Framingham Heart Study FL Server (fedprox) =====
Rounds:          10
Minimum clients: 2
Input size:      15
Save directory:  model_checkpoints
Proximal mu:     0.01

Waiting for clients to connect...



test versions completed

checked for model posioning / anomayly / DP


updated v2

working server new

In [13]:
#!/usr/bin/env python3
"""
Enhanced Federated Learning Server with Integrated Security Systems
==================================================================

This server integrates multiple security layers:
- Poisoning Detection System
- Anomaly Detection System
- Multi-layer threat assessment
- Enhanced client filtering and reputation management
"""

import flwr as fl
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import logging
import os
import sys
import json
from collections import OrderedDict
from typing import List, Tuple, Dict, Optional
from datetime import datetime

from flwr.common import parameters_to_ndarrays, ndarrays_to_parameters
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Import security modules
from poisoning_detector import PoisoningDetector
from anomaly_detector import AnomalyDetector, integrate_with_poisoning_detector

# Configure logging with Unicode support
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - EnhancedFraminghamServer - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("enhanced_server.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("EnhancedFraminghamServer")

# Enhanced server configuration
ENHANCED_SERVER_CONFIG = {
    # Training parameters
    "num_rounds": 20,
    "min_available_clients": 2,
    "min_fit_clients": 2,
    "min_evaluate_clients": 2,
    "input_size": 15,
    "save_dir": "enhanced_model_checkpoints",
    "aggregation": "fedprox",
    "proximal_mu": 0.01,
    "evaluation_data": "unseandata.csv",
    
    # Enhanced security configuration
    "enable_detection": True,
    "enable_anomaly_detection": True,
    "security_mode": "strict",  # "strict", "balanced", "permissive"
    
    # Poisoning detection parameters
    "poisoning_similarity_threshold": 0.4,
    "poisoning_outlier_threshold": 1.8,
    "poisoning_reputation_threshold": 0.6,
    "poisoning_reputation_decay": 0.9,
    
    # Anomaly detection parameters (highly sensitive to catch obvious anomalies)
    "anomaly_statistical_threshold": 2.0,      
    "anomaly_consensus_threshold": 0.2,        # VERY SENSITIVE - will catch obvious anomalies
    "anomaly_isolation_contamination": 0.15,   
    "anomaly_performance_threshold": 1.5,
    
    # Security thresholds
    "min_trusted_clients": 1,
    "max_suspicious_ratio": 0.5,  # Maximum ratio of suspicious clients to tolerate
    "critical_threat_action": "reject_all",  # "reject_all", "use_trusted_only", "continue"
    
    # Logging and monitoring
    "detailed_security_logs": True,
    "save_security_reports": True,
    "generate_visualizations": False,
}

class HeartDiseaseModel(nn.Module):
    """Enhanced heart disease prediction model."""
    def __init__(self, input_size):
        super(HeartDiseaseModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

class EnhancedModelEvaluator:
    """Enhanced model evaluator with security metrics."""
    
    def __init__(self, model_class, input_size, evaluation_data_path):
        self.model_class = model_class
        self.input_size = input_size
        self.evaluation_data_path = evaluation_data_path
        
    def evaluate_model(self, parameters, security_details=None):
        """Evaluate model with security context."""
        try:
            # Load test data
            df = pd.read_csv(self.evaluation_data_path)
            if "TenYearCHD" not in df.columns:
                logger.error("Target column 'TenYearCHD' not found in evaluation data")
                return {}
            
            X = df.drop(columns=["TenYearCHD"])
            y = df["TenYearCHD"]
            
            # Normalize features
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)
            
            # Create model and load parameters
            model = self.model_class(self.input_size)
            state_dict = OrderedDict()
            model_state_dict = model.state_dict()
            
            for i, (name, _) in enumerate(model_state_dict.items()):
                if i < len(parameters):
                    state_dict[name] = torch.tensor(parameters[i])
            
            model.load_state_dict(state_dict)
            model.eval()
            
            # Predict
            with torch.no_grad():
                X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
                predictions = model(X_tensor).numpy().flatten()
                predictions_binary = (predictions > 0.5).astype(int)
            
            # Calculate metrics
            accuracy = accuracy_score(y, predictions_binary)
            precision = precision_score(y, predictions_binary, zero_division=0)
            recall = recall_score(y, predictions_binary, zero_division=0)
            f1 = f1_score(y, predictions_binary, zero_division=0)
            
            try:
                auc = roc_auc_score(y, predictions)
            except:
                auc = 0.0
            
            metrics = {
                "accuracy": float(accuracy),
                "precision": float(precision),
                "recall": float(recall),
                "f1_score": float(f1),
                "auc": float(auc),
                "num_samples": len(y),
                "timestamp": datetime.now().isoformat()
            }
            
            # Add security context if available
            if security_details:
                metrics["security_context"] = {
                    "threat_level": security_details.get("combined_threat_level", "unknown"),
                    "trusted_clients_used": security_details.get("trusted_clients", []),
                    "suspicious_clients": security_details.get("all_suspicious", []),
                    "num_trusted": len(security_details.get("trusted_clients", [])),
                    "num_suspicious": len(security_details.get("all_suspicious", []))
                }
            
            logger.info(f"Enhanced evaluation metrics: {metrics}")
            return metrics
            
        except Exception as e:
            logger.error(f"Enhanced evaluation failed: {str(e)}")
            return {}

class SecurityThreatAssessment:
    """Centralized security threat assessment system."""
    
    @staticmethod
    def assess_combined_threat(poisoning_details, anomaly_details, num_clients):
        """Assess combined threat level from both detection systems."""
        
        # Extract key information
        poison_threat = poisoning_details.get("threat_level", "low")
        poison_suspicious = set(poisoning_details.get("suspicious_clients", []))
        
        anomaly_scores = anomaly_details.get("final_scores", {})
        anomaly_suspicious = set(cid for cid, score in anomaly_scores.items() if score > 0.5)
        
        # Calculate threat metrics
        all_suspicious = poison_suspicious | anomaly_suspicious
        high_confidence_suspicious = poison_suspicious & anomaly_suspicious
        
        suspicious_ratio = len(all_suspicious) / num_clients if num_clients > 0 else 0
        high_confidence_ratio = len(high_confidence_suspicious) / num_clients if num_clients > 0 else 0
        
        # Determine combined threat level
        if high_confidence_ratio >= 0.5 or poison_threat == "critical":
            combined_threat = "critical"
        elif high_confidence_ratio >= 0.3 or suspicious_ratio >= 0.6 or poison_threat == "high":
            combined_threat = "high"
        elif suspicious_ratio >= 0.3 or poison_threat == "medium":
            combined_threat = "medium"
        else:
            combined_threat = "low"
        
        return {
            "combined_threat_level": combined_threat,
            "suspicious_ratio": suspicious_ratio,
            "high_confidence_ratio": high_confidence_ratio,
            "poison_suspicious": list(poison_suspicious),
            "anomaly_suspicious": list(anomaly_suspicious),
            "all_suspicious": list(all_suspicious),
            "high_confidence_suspicious": list(high_confidence_suspicious),
            "threat_factors": {
                "poisoning_threat": poison_threat,
                "anomaly_threat": "high" if len(anomaly_suspicious) >= num_clients * 0.4 else "medium" if anomaly_suspicious else "low",
                "consensus_threat": "high" if high_confidence_suspicious else "low"
            }
        }

def enhanced_federated_averaging(results):
    """Enhanced federated averaging with security logging."""
    weights_results = [(parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples) for _, fit_res in results]
    total_examples = sum([num_examples for _, num_examples in weights_results])
    
    logger.info(f"Enhanced aggregation from {len(weights_results)} clients with {total_examples} total examples")

    if total_examples == 0:
        return None

    weighted_weights = []
    for weights, num_examples in weights_results:
        if num_examples == 0:
            continue
        weight = num_examples / total_examples
        if not weighted_weights:
            weighted_weights = [layer * weight for layer in weights]
        else:
            for i, layer in enumerate(weights):
                weighted_weights[i] += layer * weight

    return weighted_weights

class EnhancedFedProxStrategy(fl.server.strategy.FedAvg):
    """Enhanced FedProx strategy with integrated multi-layer security."""
    
    def __init__(self, initial_parameters, save_dir=None, evaluator=None):
        super().__init__(
            fraction_fit=1.0,
            fraction_evaluate=1.0,
            min_fit_clients=ENHANCED_SERVER_CONFIG["min_fit_clients"],
            min_evaluate_clients=ENHANCED_SERVER_CONFIG["min_evaluate_clients"],
            min_available_clients=ENHANCED_SERVER_CONFIG["min_available_clients"],
        )
        
        self.initial_parameters = initial_parameters
        self.save_dir = save_dir or ENHANCED_SERVER_CONFIG["save_dir"]
        self.metrics_history = []
        self.security_history = []
        self.evaluator = evaluator
        
        # Initialize security systems
        self._initialize_security_systems()
        
        # Create directories
        os.makedirs(self.save_dir, exist_ok=True)
        os.makedirs(os.path.join(self.save_dir, "security_logs"), exist_ok=True)
        os.makedirs(os.path.join(self.save_dir, "security_reports"), exist_ok=True)
        
        logger.info("Enhanced FedProx Strategy with Multi-Layer Security initialized")
    
    def _initialize_security_systems(self):
        """Initialize all security detection systems."""
        
        # Initialize poisoning detector
        if ENHANCED_SERVER_CONFIG["enable_detection"]:
            self.poisoning_detector = PoisoningDetector(
                similarity_threshold=ENHANCED_SERVER_CONFIG["poisoning_similarity_threshold"],
                outlier_threshold=ENHANCED_SERVER_CONFIG["poisoning_outlier_threshold"],
                reputation_threshold=ENHANCED_SERVER_CONFIG["poisoning_reputation_threshold"],
                reputation_decay=ENHANCED_SERVER_CONFIG["poisoning_reputation_decay"],
                save_logs=True,
                log_dir=os.path.join(self.save_dir, "security_logs", "poisoning")
            )
            logger.info("[SECURITY] Poisoning detection system enabled")
        else:
            self.poisoning_detector = None
            logger.info("[SECURITY] Poisoning detection disabled")
        
        # Initialize anomaly detector
        if ENHANCED_SERVER_CONFIG["enable_anomaly_detection"]:
            self.anomaly_detector = AnomalyDetector(
                statistical_threshold=ENHANCED_SERVER_CONFIG["anomaly_statistical_threshold"],
                consensus_threshold=ENHANCED_SERVER_CONFIG["anomaly_consensus_threshold"],
                isolation_contamination=ENHANCED_SERVER_CONFIG["anomaly_isolation_contamination"],
                performance_threshold=ENHANCED_SERVER_CONFIG["anomaly_performance_threshold"],
                save_logs=True,
                log_dir=os.path.join(self.save_dir, "security_logs", "anomaly")
            )
            logger.info("[SECURITY] Anomaly detection system enabled")
        else:
            self.anomaly_detector = None
            logger.info("[SECURITY] Anomaly detection disabled")
        
        # Create combined detector if both systems are enabled
        if self.poisoning_detector and self.anomaly_detector:
            self.combined_detector = integrate_with_poisoning_detector(
                self.poisoning_detector, self.anomaly_detector
            )
            logger.info("[SECURITY] Combined multi-layer detection enabled")
        else:
            self.combined_detector = None
            logger.info("[SECURITY] Combined detection disabled - using individual systems")

    def initialize_parameters(self, client_manager):
        return self.initial_parameters

    def aggregate_fit(self, server_round, results, failures):
        """Enhanced aggregation with multi-layer security analysis."""
        
        print(f"\n{'='*80}")
        print(f"🔄 ENHANCED ROUND {server_round} AGGREGATION")
        print(f"{'='*80}")
        print(f"Received {len(results)} client results")
        
        if not results:
            logger.warning(f"Round {server_round}: No results to aggregate!")
            return None, {}

        # Extract client data
        print(f"📊 Extracting client data...")
        client_updates = {}
        client_metrics = {}
        
        for i, (client_proxy, fit_res) in enumerate(results):
            # Enhanced client ID extraction with fallback
            client_id = None
            
            # Try to get client ID from metrics first
            if fit_res.metrics and 'client_id' in fit_res.metrics:
                client_id = str(fit_res.metrics['client_id'])
            
            # Try to get from proxy
            elif hasattr(client_proxy, 'cid') and client_proxy.cid:
                client_id = str(client_proxy.cid)
            
            # Fallback: use index with additional identifier
            if client_id is None or client_id in client_updates:
                # If duplicate or missing, create unique ID
                client_id = f"client_{i}_{hash(str(fit_res.parameters))[:6]}"
            
            print(f"  Processing {client_id}...")
            
            # Convert parameters to numpy arrays
            weights = parameters_to_ndarrays(fit_res.parameters)
            client_updates[client_id] = weights
            
            # Extract metrics
            metrics = fit_res.metrics if fit_res.metrics else {}
            client_metrics[client_id] = {
                'accuracy': metrics.get('accuracy', 0.0),
                'loss': metrics.get('loss', float('inf')),
                'client_id': client_id,
                'poisoning_active': metrics.get('poisoning_active', False),
                **metrics
            }
            
            # Debug info
            total_norm = sum(np.linalg.norm(w) for w in weights)
            print(f"    Metrics: acc={client_metrics[client_id]['accuracy']:.3f}, "
                  f"loss={client_metrics[client_id]['loss']:.3f}, "
                  f"norm={total_norm:.6f}")
            
            if client_metrics[client_id]['poisoning_active']:
                print(f"    🦠 {client_id} reports active poisoning!")

        # Run enhanced security analysis
        security_results = self._run_enhanced_security_analysis(
            client_updates, client_metrics, server_round
        )
        
        # Determine aggregation strategy based on security assessment
        aggregation_decision = self._make_aggregation_decision(
            security_results, len(results)
        )
        
        print(f"\n🛡️ AGGREGATION DECISION: {aggregation_decision['action'].upper()}")
        print(f"   Rationale: {aggregation_decision['rationale']}")
        
        # Filter clients based on security decision
        if aggregation_decision["action"] == "reject_all":
            logger.critical(f"[SECURITY] Round {server_round}: REJECTING ALL CLIENTS due to critical threat")
            return None, {"error": "Critical security threat detected", "security_details": security_results}
        
        elif aggregation_decision["action"] == "use_trusted_only":
            trusted_clients = aggregation_decision["trusted_clients"]
            print(f"✅ Using {len(trusted_clients)} trusted clients for aggregation")
            
            filtered_results = []
            for client_proxy, fit_res in results:
                # Get client ID
                if fit_res.metrics and 'client_id' in fit_res.metrics:
                    client_id = str(fit_res.metrics['client_id'])
                elif hasattr(client_proxy, 'cid') and client_proxy.cid:
                    client_id = str(client_proxy.cid)
                else:
                    for j, (cp, fr) in enumerate(results):
                        if cp == client_proxy and fr == fit_res:
                            client_id = f"client_{j}"
                            break
                
                if client_id in trusted_clients:
                    filtered_results.append((client_proxy, fit_res))
        
        else:  # use_all
            print(f"⚠️ Using all clients (security override or low threat)")
            filtered_results = results

        if not filtered_results:
            logger.error(f"[SECURITY] Round {server_round}: No trusted clients available")
            return None, {"error": "No trusted clients available", "security_details": security_results}

        # Perform enhanced aggregation
        print(f"\n📊 Performing enhanced aggregation with {len(filtered_results)} clients...")
        aggregated_ndarrays = enhanced_federated_averaging(filtered_results)

        if aggregated_ndarrays is None:
            print(f"❌ Enhanced aggregation failed!")
            return None, {}

        aggregated_parameters = ndarrays_to_parameters(aggregated_ndarrays)

        # Calculate aggregation metrics
        if filtered_results:
            accuracies = [fr.metrics.get("accuracy", 0.0) for _, fr in filtered_results if fr.metrics]
            losses = [fr.metrics.get("loss", 0.0) for _, fr in filtered_results if fr.metrics]
            avg_accuracy = np.mean(accuracies) if accuracies else 0.0
            avg_loss = np.mean(losses) if losses else 0.0
        else:
            avg_accuracy = 0.0
            avg_loss = 0.0

        # Create enhanced metrics
        enhanced_metrics = {
            "round": server_round,
            "accuracy": float(avg_accuracy),
            "loss": float(avg_loss),
            "security_results": security_results,
            "aggregation_decision": aggregation_decision,
            "trusted_clients_count": len(aggregation_decision.get("trusted_clients", [])),
            "suspicious_clients_count": len(security_results.get("all_suspicious", [])),
            "aggregated_clients": len(filtered_results),
            "total_clients": len(results),
            "security_score": self._calculate_security_score(security_results),
            "timestamp": datetime.now().isoformat()
        }

        # Save enhanced results
        self._save_enhanced_model(server_round, aggregated_ndarrays, security_results)
        self._save_enhanced_metrics(enhanced_metrics)
        self._generate_security_report(server_round, security_results, enhanced_metrics)

        # Enhanced logging
        self._log_enhanced_results(server_round, enhanced_metrics)

        return aggregated_parameters, enhanced_metrics

    def _run_enhanced_security_analysis(self, client_updates, client_metrics, server_round):
        """Run comprehensive security analysis using all available detection systems."""
        
        print(f"\n🔍 ENHANCED SECURITY ANALYSIS - ROUND {server_round}")
        print(f"{'='*60}")
        
        security_results = {
            "round": server_round,
            "num_clients": len(client_updates),
            "timestamp": datetime.now().isoformat()
        }
        
        # Run poisoning detection
        if self.poisoning_detector and len(client_updates) >= 2:
            print("🔍 Running Poisoning Detection...")
            try:
                is_safe_poison, trusted_poison, poison_details = self.poisoning_detector.analyze_round(
                    client_updates, client_metrics, round_num=server_round
                )
                security_results["poisoning_detection"] = poison_details
                security_results["poisoning_safe"] = is_safe_poison
                print(f"   ✅ Poisoning detection completed")
            except Exception as e:
                logger.error(f"[SECURITY] Poisoning detection failed: {str(e)}")
                security_results["poisoning_detection"] = {"error": str(e)}
                security_results["poisoning_safe"] = True
                trusted_poison = list(client_updates.keys())
        else:
            print("⚠️ Poisoning detection skipped")
            security_results["poisoning_detection"] = {"skipped": "insufficient clients or disabled"}
            security_results["poisoning_safe"] = True
            trusted_poison = list(client_updates.keys())
        
        # Run anomaly detection
        if self.anomaly_detector and len(client_updates) >= 2:
            print("🔍 Running Anomaly Detection...")
            try:
                anomaly_scores, anomaly_details = self.anomaly_detector.detect_anomalies(
                    client_updates, client_metrics, round_num=server_round
                )
                security_results["anomaly_detection"] = anomaly_details
                security_results["anomaly_scores"] = anomaly_scores
                print(f"   ✅ Anomaly detection completed")
            except Exception as e:
                logger.error(f"[SECURITY] Anomaly detection failed: {str(e)}")
                security_results["anomaly_detection"] = {"error": str(e)}
                security_results["anomaly_scores"] = {cid: 0.0 for cid in client_updates.keys()}
        else:
            print("⚠️ Anomaly detection skipped")
            security_results["anomaly_detection"] = {"skipped": "insufficient clients or disabled"}
            security_results["anomaly_scores"] = {cid: 0.0 for cid in client_updates.keys()}
        
        # Combine results using threat assessment
        print("🔍 Performing Combined Threat Assessment...")
        poison_details = security_results.get("poisoning_detection", {})
        anomaly_details = security_results.get("anomaly_detection", {})
        
        combined_assessment = SecurityThreatAssessment.assess_combined_threat(
            poison_details, anomaly_details, len(client_updates)
        )
        security_results.update(combined_assessment)
        
        # Determine final trusted clients
        poison_suspicious = set(poison_details.get("suspicious_clients", []))
        anomaly_suspicious = set(cid for cid, score in security_results["anomaly_scores"].items() if score > 0.5)
        
        all_clients = set(client_updates.keys())
        all_suspicious = poison_suspicious | anomaly_suspicious
        trusted_clients = list(all_clients - all_suspicious)
        
        security_results["trusted_clients"] = trusted_clients
        
        print(f"🛡️ ENHANCED SECURITY SUMMARY:")
        print(f"   Combined Threat Level: {security_results['combined_threat_level'].upper()}")
        print(f"   Poisoning Suspicious: {list(poison_suspicious)}")
        print(f"   Anomaly Suspicious: {list(anomaly_suspicious)}")
        print(f"   All Suspicious: {list(all_suspicious)}")
        print(f"   Trusted Clients: {trusted_clients}")
        
        return security_results
    
    def _make_aggregation_decision(self, security_results, total_clients):
        """Make intelligent aggregation decision based on security analysis and configuration."""
        
        threat_level = security_results.get("combined_threat_level", "unknown")
        trusted_clients = security_results.get("trusted_clients", [])
        suspicious_clients = security_results.get("all_suspicious", [])
        
        security_mode = ENHANCED_SERVER_CONFIG["security_mode"]
        min_trusted = ENHANCED_SERVER_CONFIG["min_trusted_clients"]
        max_suspicious_ratio = ENHANCED_SERVER_CONFIG["max_suspicious_ratio"]
        critical_action = ENHANCED_SERVER_CONFIG["critical_threat_action"]
        
        suspicious_ratio = len(suspicious_clients) / total_clients if total_clients > 0 else 0
        
        # Decision logic based on threat level and security mode
        if threat_level == "critical":
            if critical_action == "reject_all":
                return {
                    "action": "reject_all",
                    "rationale": f"Critical threat detected, action: {critical_action}",
                    "trusted_clients": trusted_clients
                }
            elif critical_action == "use_trusted_only" and len(trusted_clients) >= min_trusted:
                return {
                    "action": "use_trusted_only", 
                    "rationale": f"Critical threat but {len(trusted_clients)} trusted clients available",
                    "trusted_clients": trusted_clients
                }
            else:
                return {
                    "action": "reject_all",
                    "rationale": f"Critical threat and insufficient trusted clients ({len(trusted_clients)} < {min_trusted})",
                    "trusted_clients": trusted_clients
                }
        
        elif threat_level == "high":
            if security_mode == "strict":
                if len(trusted_clients) >= min_trusted:
                    return {
                        "action": "use_trusted_only",
                        "rationale": f"High threat in strict mode, using {len(trusted_clients)} trusted clients",
                        "trusted_clients": trusted_clients
                    }
                else:
                    return {
                        "action": "reject_all", 
                        "rationale": f"High threat and insufficient trusted clients in strict mode",
                        "trusted_clients": trusted_clients
                    }
            else:  # balanced or permissive
                return {
                    "action": "use_trusted_only" if len(trusted_clients) >= min_trusted else "use_all",
                    "rationale": f"High threat in {security_mode} mode",
                    "trusted_clients": trusted_clients
                }
        
        elif threat_level == "medium":
            if security_mode == "strict" and suspicious_ratio > max_suspicious_ratio:
                return {
                    "action": "use_trusted_only" if len(trusted_clients) >= min_trusted else "reject_all",
                    "rationale": f"Medium threat exceeds suspicious ratio threshold in strict mode",
                    "trusted_clients": trusted_clients
                }
            elif suspicious_ratio > max_suspicious_ratio * 1.5:  # Very high suspicious ratio
                return {
                    "action": "use_trusted_only" if len(trusted_clients) >= min_trusted else "use_all",
                    "rationale": f"Medium threat with very high suspicious ratio",
                    "trusted_clients": trusted_clients
                }
            else:
                return {
                    "action": "use_all",
                    "rationale": f"Medium threat within acceptable parameters",
                    "trusted_clients": trusted_clients
                }
        
        else:  # low threat or unknown
            if security_mode == "strict" and suspicious_clients:
                return {
                    "action": "use_trusted_only",
                    "rationale": f"Low threat but strict mode with suspicious clients detected",
                    "trusted_clients": trusted_clients
                }
            else:
                return {
                    "action": "use_all",
                    "rationale": f"Low threat level",
                    "trusted_clients": trusted_clients
                }
    
    def _calculate_security_score(self, security_results):
        """Calculate overall security score for the round."""
        threat_level = security_results.get("combined_threat_level", "unknown")
        
        threat_scores = {
            "low": 0.9,
            "medium": 0.7,
            "high": 0.4,
            "critical": 0.1,
            "unknown": 0.5
        }
        
        base_score = threat_scores.get(threat_level, 0.5)
        
        # Adjust based on ratios
        suspicious_ratio = security_results.get("suspicious_ratio", 0)
        high_confidence_ratio = security_results.get("high_confidence_ratio", 0)
        
        # Penalty for high suspicious ratios
        ratio_penalty = min(0.3, suspicious_ratio * 0.5 + high_confidence_ratio * 0.3)
        
        final_score = max(0.0, base_score - ratio_penalty)
        return round(final_score, 3)
    
    def _save_enhanced_model(self, round_num, parameters, security_results):
        """Save model with enhanced security metadata."""
        try:
            # Save model parameters
            model_path = os.path.join(self.save_dir, f"enhanced_model_round_{round_num}.pt")
            params_path = os.path.join(self.save_dir, f"enhanced_params_round_{round_num}.npz")
            security_path = os.path.join(self.save_dir, f"security_round_{round_num}.json")
            
            # Create model and save
            model = HeartDiseaseModel(input_size=ENHANCED_SERVER_CONFIG["input_size"])
            state_dict = OrderedDict()
            model_state_dict = model.state_dict()

            for i, (name, _) in enumerate(model_state_dict.items()):
                if i < len(parameters):
                    state_dict[name] = torch.tensor(parameters[i])

            model.load_state_dict(state_dict)
            torch.save(model.state_dict(), model_path)
            
            # Save parameters
            np.savez(params_path, *parameters)
            
            # Save security details with enhanced metadata
            enhanced_security = {
                **security_results,
                "model_path": model_path,
                "params_path": params_path,
                "round": round_num,
                "saved_timestamp": datetime.now().isoformat()
            }
            
            with open(security_path, 'w') as f:
                json.dump(enhanced_security, f, indent=2, default=str)

            logger.info(f"[SAVED] Enhanced model and security data for round {round_num}")

        except Exception as e:
            logger.error(f"Failed to save enhanced model: {str(e)}", exc_info=True)
    
    def _save_enhanced_metrics(self, enhanced_metrics):
        """Save enhanced metrics with security information."""
        try:
            self.metrics_history.append(enhanced_metrics)
            self.security_history.append(enhanced_metrics.get("security_results", {}))
            
            # Save comprehensive metrics
            metrics_path = os.path.join(self.save_dir, "enhanced_metrics_history.json")
            with open(metrics_path, 'w') as f:
                json.dump(self.metrics_history, f, indent=2, default=str)
            
            # Save security-focused history
            security_path = os.path.join(self.save_dir, "security_history.json")
            with open(security_path, 'w') as f:
                json.dump(self.security_history, f, indent=2, default=str)
                
            logger.info(f"[SAVED] Enhanced metrics and security history")
            
        except Exception as e:
            logger.error(f"Failed to save enhanced metrics: {e}")
    
    def _generate_security_report(self, round_num, security_results, enhanced_metrics):
        """Generate detailed security report for the round."""
        if not ENHANCED_SERVER_CONFIG["save_security_reports"]:
            return
        
        try:
            report = {
                "report_metadata": {
                    "round": round_num,
                    "timestamp": datetime.now().isoformat(),
                    "report_type": "security_analysis",
                    "version": "1.0"
                },
                "executive_summary": {
                    "threat_level": security_results.get("combined_threat_level", "unknown"),
                    "security_score": enhanced_metrics.get("security_score", 0.0),
                    "total_clients": security_results.get("num_clients", 0),
                    "trusted_clients": len(security_results.get("trusted_clients", [])),
                    "suspicious_clients": len(security_results.get("all_suspicious", [])),
                    "aggregation_action": enhanced_metrics.get("aggregation_decision", {}).get("action", "unknown")
                },
                "detailed_analysis": {
                    "poisoning_detection": security_results.get("poisoning_detection", {}),
                    "anomaly_detection": security_results.get("anomaly_detection", {}),
                    "threat_assessment": {
                        "combined_threat_level": security_results.get("combined_threat_level"),
                        "threat_factors": security_results.get("threat_factors", {}),
                        "suspicious_ratio": security_results.get("suspicious_ratio", 0),
                        "high_confidence_ratio": security_results.get("high_confidence_ratio", 0)
                    }
                },
                "client_analysis": {
                    "trusted_clients": security_results.get("trusted_clients", []),
                    "poison_suspicious": security_results.get("poison_suspicious", []),
                    "anomaly_suspicious": security_results.get("anomaly_suspicious", []),
                    "high_confidence_suspicious": security_results.get("high_confidence_suspicious", []),
                    "anomaly_scores": security_results.get("anomaly_scores", {})
                },
                "aggregation_details": enhanced_metrics.get("aggregation_decision", {}),
                "performance_metrics": {
                    "accuracy": enhanced_metrics.get("accuracy", 0.0),
                    "loss": enhanced_metrics.get("loss", 0.0),
                    "aggregated_clients": enhanced_metrics.get("aggregated_clients", 0)
                }
            }
            
            # Save report
            report_path = os.path.join(self.save_dir, "security_reports", f"security_report_round_{round_num}.json")
            with open(report_path, 'w') as f:
                json.dump(report, f, indent=2, default=str)
            
            # Generate summary report if it's the final round
            if round_num == ENHANCED_SERVER_CONFIG["num_rounds"]:
                self._generate_final_security_summary()
                
        except Exception as e:
            logger.error(f"Failed to generate security report: {e}")
    
    def _generate_final_security_summary(self):
        """Generate comprehensive security summary for all rounds."""
        try:
            if not self.security_history:
                return
            
            # Analyze trends across rounds
            threat_levels = [round_data.get("combined_threat_level", "unknown") for round_data in self.security_history]
            security_scores = [metrics.get("security_score", 0.0) for metrics in self.metrics_history]
            
            # Calculate statistics
            threat_level_counts = {level: threat_levels.count(level) for level in set(threat_levels)}
            avg_security_score = np.mean(security_scores) if security_scores else 0.0
            min_security_score = min(security_scores) if security_scores else 0.0
            
            # Identify most problematic rounds
            problematic_rounds = []
            for i, metrics in enumerate(self.metrics_history):
                if metrics.get("security_score", 1.0) < 0.5:
                    problematic_rounds.append({
                        "round": metrics.get("round", i+1),
                        "threat_level": self.security_history[i].get("combined_threat_level", "unknown"),
                        "security_score": metrics.get("security_score", 0.0),
                        "suspicious_clients": self.security_history[i].get("all_suspicious", [])
                    })
            
            # Client reputation analysis
            all_clients = set()
            client_suspicious_counts = {}
            
            for round_data in self.security_history:
                suspicious = round_data.get("all_suspicious", [])
                for client in suspicious:
                    all_clients.add(client)
                    client_suspicious_counts[client] = client_suspicious_counts.get(client, 0) + 1
            
            summary = {
                "summary_metadata": {
                    "total_rounds": len(self.security_history),
                    "analysis_timestamp": datetime.now().isoformat(),
                    "security_config": {
                        "security_mode": ENHANCED_SERVER_CONFIG["security_mode"],
                        "detection_enabled": ENHANCED_SERVER_CONFIG["enable_detection"],
                        "anomaly_detection_enabled": ENHANCED_SERVER_CONFIG["enable_anomaly_detection"]
                    }
                },
                "overall_statistics": {
                    "threat_level_distribution": threat_level_counts,
                    "average_security_score": round(avg_security_score, 3),
                    "minimum_security_score": round(min_security_score, 3),
                    "problematic_rounds_count": len(problematic_rounds),
                    "total_unique_clients": len(all_clients)
                },
                "problematic_rounds": problematic_rounds,
                "client_reputation_analysis": {
                    "frequently_suspicious_clients": {
                        client: count for client, count in client_suspicious_counts.items()
                        if count >= len(self.security_history) * 0.3  # Suspicious in 30%+ of rounds
                    },
                    "client_suspicious_counts": client_suspicious_counts
                },
                "security_trends": {
                    "security_scores_by_round": [
                        {"round": i+1, "score": score} 
                        for i, score in enumerate(security_scores)
                    ],
                    "threat_levels_by_round": [
                        {"round": i+1, "threat_level": level}
                        for i, level in enumerate(threat_levels)
                    ]
                },
                "recommendations": self._generate_security_recommendations(
                    threat_level_counts, avg_security_score, client_suspicious_counts
                )
            }
            
            # Save final summary
            summary_path = os.path.join(self.save_dir, "final_security_summary.json")
            with open(summary_path, 'w') as f:
                json.dump(summary, f, indent=2, default=str)
            
            logger.info(f"[REPORT] Generated final security summary")
            print(f"\n📊 FINAL SECURITY SUMMARY:")
            print(f"   Average Security Score: {avg_security_score:.3f}")
            print(f"   Problematic Rounds: {len(problematic_rounds)}")
            print(f"   Threat Distribution: {threat_level_counts}")
            
        except Exception as e:
            logger.error(f"Failed to generate final security summary: {e}")
    
    def _generate_security_recommendations(self, threat_counts, avg_score, client_suspicious_counts):
        """Generate security recommendations based on analysis."""
        recommendations = []
        
        # Score-based recommendations
        if avg_score < 0.5:
            recommendations.append({
                "type": "critical",
                "message": "Average security score is very low. Consider stricter security policies.",
                "action": "Increase security_mode to 'strict' and lower detection thresholds."
            })
        elif avg_score < 0.7:
            recommendations.append({
                "type": "warning", 
                "message": "Security score indicates moderate risk. Monitor closely.",
                "action": "Consider enabling additional detection methods."
            })
        
        # Threat level recommendations
        critical_count = threat_counts.get("critical", 0)
        high_count = threat_counts.get("high", 0)
        
        if critical_count > 0:
            recommendations.append({
                "type": "critical",
                "message": f"Critical threats detected in {critical_count} rounds.",
                "action": "Review client authentication and implement stricter filtering."
            })
        
        if high_count > len(threat_counts) * 0.3:
            recommendations.append({
                "type": "warning",
                "message": f"High threat levels detected frequently ({high_count} rounds).",
                "action": "Consider implementing client whitelisting or reputation-based filtering."
            })
        
        # Client-specific recommendations
        frequent_offenders = [
            client for client, count in client_suspicious_counts.items()
            if count >= len(self.security_history) * 0.5
        ]
        
        if frequent_offenders:
            recommendations.append({
                "type": "action_required",
                "message": f"Clients frequently flagged as suspicious: {frequent_offenders}",
                "action": "Consider blacklisting or requiring additional verification for these clients."
            })
        
        return recommendations
    
    def _log_enhanced_results(self, round_num, enhanced_metrics):
        """Enhanced logging with security context."""
        
        security_results = enhanced_metrics.get("security_results", {})
        threat_level = security_results.get("combined_threat_level", "unknown")
        security_score = enhanced_metrics.get("security_score", 0.0)
        
        # Performance metrics
        accuracy = enhanced_metrics.get("accuracy", 0.0)
        loss = enhanced_metrics.get("loss", 0.0)
        
        # Security metrics
        trusted_count = enhanced_metrics.get("trusted_clients_count", 0)
        suspicious_count = enhanced_metrics.get("suspicious_clients_count", 0)
        aggregated_count = enhanced_metrics.get("aggregated_clients", 0)
        total_count = enhanced_metrics.get("total_clients", 0)
        
        print(f"\n[ENHANCED] ROUND {round_num} RESULTS:")
        print(f"   Performance: Accuracy={accuracy:.4f}, Loss={loss:.4f}")
        print(f"   Security: Score={security_score:.3f}, Threat={threat_level.upper()}")
        print(f"   Clients: {aggregated_count}/{total_count} aggregated ({trusted_count} trusted, {suspicious_count} suspicious)")
        
        # Log based on threat level
        if threat_level == "critical":
            logger.critical(f"[SECURITY] Round {round_num}: CRITICAL THREAT DETECTED!")
            logger.critical(f"  Security Score: {security_score:.3f}")
            logger.critical(f"  Suspicious Clients: {security_results.get('all_suspicious', [])}")
        elif threat_level == "high":
            logger.error(f"[SECURITY] Round {round_num}: High threat level detected")
            logger.error(f"  Security Score: {security_score:.3f}")
            logger.error(f"  Suspicious Clients: {security_results.get('all_suspicious', [])}")
        elif threat_level == "medium":
            logger.warning(f"[SECURITY] Round {round_num}: Medium threat level")
            logger.warning(f"  Security Score: {security_score:.3f}")
        else:
            logger.info(f"[SECURITY] Round {round_num}: Low threat level (Score: {security_score:.3f})")
        
        # Performance logging
        logger.info(f"[PERFORMANCE] Round {round_num}: Accuracy={accuracy:.4f}, Loss={loss:.4f}")
        logger.info(f"[AGGREGATION] Round {round_num}: Used {aggregated_count}/{total_count} clients")

    def aggregate_evaluate(self, server_round, results, failures):
        """Enhanced evaluation aggregation."""
        if not results:
            return None, {}

        logger.info(f"Round {server_round}: Aggregating evaluation from {len(results)} clients")

        accuracies = [r.metrics.get("accuracy", 0.0) for _, r in results if r.metrics]
        losses = [r.loss for _, r in results]

        avg_accuracy = sum(accuracies) / len(accuracies) if accuracies else 0.0
        avg_loss = sum(losses) / len(losses) if losses else 0.0

        metrics = {
            "accuracy": float(avg_accuracy),
            "loss": float(avg_loss),
            "evaluation_timestamp": datetime.now().isoformat()
        }

        logger.info(f"Round {server_round} evaluation - Accuracy: {avg_accuracy:.4f}, Loss: {avg_loss:.4f}")

        return avg_loss, metrics

    def final_evaluation(self):
        """Enhanced final evaluation with security context."""
        if self.evaluator and self.metrics_history:
            try:
                final_round = self.metrics_history[-1]["round"]
                params_path = os.path.join(self.save_dir, f"enhanced_params_round_{final_round}.npz")
                
                if os.path.exists(params_path):
                    loaded_params = np.load(params_path)
                    final_params = [loaded_params[f"arr_{i}"] for i in range(len(loaded_params.files))]

                    logger.info("\n" + "=" * 60)
                    logger.info("ENHANCED FINAL MODEL EVALUATION")
                    logger.info("=" * 60)

                    # Get security context for final evaluation
                    final_security = self.metrics_history[-1].get("security_results", {})
                    
                    final_metrics = self.evaluator.evaluate_model(final_params, final_security)

                    # Save enhanced final evaluation
                    final_eval_path = os.path.join(self.save_dir, "enhanced_final_evaluation.json")
                    with open(final_eval_path, "w") as f:
                        json.dump(final_metrics, f, indent=2, default=str)

                    return final_metrics
                    
            except Exception as e:
                logger.error(f"Enhanced final evaluation failed: {str(e)}")
        return {}

    def get_security_summary(self):
        """Get comprehensive security summary."""
        if not self.security_history:
            return pd.DataFrame()
        
        summary_data = []
        for i, (security_data, metrics_data) in enumerate(zip(self.security_history, self.metrics_history)):
            summary_data.append({
                'round': security_data.get('round', i+1),
                'threat_level': security_data.get('combined_threat_level', 'unknown'),
                'security_score': metrics_data.get('security_score', 0.0),
                'trusted_clients': len(security_data.get('trusted_clients', [])),
                'suspicious_clients': len(security_data.get('all_suspicious', [])),
                'total_clients': security_data.get('num_clients', 0),
                'accuracy': metrics_data.get('accuracy', 0.0),
                'loss': metrics_data.get('loss', 0.0),
                'aggregation_action': metrics_data.get('aggregation_decision', {}).get('action', 'unknown')
            })
        
        return pd.DataFrame(summary_data)

def start_enhanced_server(
    num_rounds=None,
    min_clients=None,
    input_size=None,
    save_dir=None,
    aggregation=None,
    proximal_mu=None,
    evaluation_data=None,
    security_mode=None,
    enable_detection=None,
    enable_anomaly_detection=None
):
    """Start enhanced federated learning server with integrated security."""
    
    # Update configuration
    if num_rounds is not None:
        ENHANCED_SERVER_CONFIG["num_rounds"] = num_rounds
    if min_clients is not None:
        ENHANCED_SERVER_CONFIG["min_available_clients"] = min_clients
        ENHANCED_SERVER_CONFIG["min_fit_clients"] = min_clients
        ENHANCED_SERVER_CONFIG["min_evaluate_clients"] = min_clients
    if input_size is not None:
        ENHANCED_SERVER_CONFIG["input_size"] = input_size
    if save_dir is not None:
        ENHANCED_SERVER_CONFIG["save_dir"] = save_dir
    if aggregation is not None:
        ENHANCED_SERVER_CONFIG["aggregation"] = aggregation
    if proximal_mu is not None:
        ENHANCED_SERVER_CONFIG["proximal_mu"] = proximal_mu
    if evaluation_data is not None:
        ENHANCED_SERVER_CONFIG["evaluation_data"] = evaluation_data
    if security_mode is not None:
        ENHANCED_SERVER_CONFIG["security_mode"] = security_mode
    if enable_detection is not None:
        ENHANCED_SERVER_CONFIG["enable_detection"] = enable_detection
    if enable_anomaly_detection is not None:
        ENHANCED_SERVER_CONFIG["enable_anomaly_detection"] = enable_anomaly_detection

    # Create directories
    os.makedirs(ENHANCED_SERVER_CONFIG["save_dir"], exist_ok=True)

    # Initialize model
    model = HeartDiseaseModel(input_size=ENHANCED_SERVER_CONFIG["input_size"])
    initial_params = [val.detach().cpu().numpy() for val in model.parameters()]
    initial_parameters = ndarrays_to_parameters(initial_params)

    # Initialize evaluator
    evaluator = None
    if ENHANCED_SERVER_CONFIG["evaluation_data"] and os.path.exists(ENHANCED_SERVER_CONFIG["evaluation_data"]):
        evaluator = EnhancedModelEvaluator(
            model_class=HeartDiseaseModel,
            input_size=ENHANCED_SERVER_CONFIG["input_size"],
            evaluation_data_path=ENHANCED_SERVER_CONFIG["evaluation_data"],
        )
        logger.info(f"Initialized enhanced evaluator with data: {ENHANCED_SERVER_CONFIG['evaluation_data']}")
    else:
        logger.warning(f"Evaluation data not found: {ENHANCED_SERVER_CONFIG['evaluation_data']}")

    # Create enhanced strategy
    strategy = EnhancedFedProxStrategy(
        initial_parameters=initial_parameters,
        save_dir=ENHANCED_SERVER_CONFIG["save_dir"],
        evaluator=evaluator,
    )

    # Display enhanced server information
    print(f"\n{'='*80}")
    print(f"[ENHANCED] FRAMINGHAM FL SERVER WITH INTEGRATED SECURITY")
    print(f"{'='*80}")
    print(f"Training Configuration:")
    print(f"  Rounds:              {ENHANCED_SERVER_CONFIG['num_rounds']}")
    print(f"  Minimum clients:     {ENHANCED_SERVER_CONFIG['min_available_clients']}")
    print(f"  Input size:          {ENHANCED_SERVER_CONFIG['input_size']}")
    print(f"  Aggregation:         {ENHANCED_SERVER_CONFIG['aggregation']}")
    print(f"  Save directory:      {ENHANCED_SERVER_CONFIG['save_dir']}")
    print(f"  Evaluation data:     {ENHANCED_SERVER_CONFIG['evaluation_data']}")
    if ENHANCED_SERVER_CONFIG["aggregation"] == "fedprox":
        print(f"  Proximal mu:         {ENHANCED_SERVER_CONFIG['proximal_mu']}")
    
    print(f"\nSecurity Configuration:")
    print(f"  Security Mode:       {ENHANCED_SERVER_CONFIG['security_mode'].upper()}")
    print(f"  Poisoning Detection: {'ENABLED' if ENHANCED_SERVER_CONFIG['enable_detection'] else 'DISABLED'}")
    print(f"  Anomaly Detection:   {'ENABLED' if ENHANCED_SERVER_CONFIG['enable_anomaly_detection'] else 'DISABLED'}")
    print(f"  Min Trusted Clients: {ENHANCED_SERVER_CONFIG['min_trusted_clients']}")
    print(f"  Max Suspicious Ratio: {ENHANCED_SERVER_CONFIG['max_suspicious_ratio']}")
    print(f"  Critical Threat Action: {ENHANCED_SERVER_CONFIG['critical_threat_action']}")
    
    print(f"\nDetection Thresholds:")
    print(f"  Poisoning Similarity: {ENHANCED_SERVER_CONFIG['poisoning_similarity_threshold']}")
    print(f"  Anomaly Statistical:  {ENHANCED_SERVER_CONFIG['anomaly_statistical_threshold']}")
    print(f"  Anomaly Consensus:    {ENHANCED_SERVER_CONFIG['anomaly_consensus_threshold']}")
    
    print(f"{'='*80}")
    print(f"🔍 Enhanced security systems initialized and ready")
    print(f"📊 Comprehensive logging and monitoring enabled")
    print(f"🛡️ Multi-layer threat detection active")
    print(f"\nWaiting for clients to connect...\n")

    # Server configuration
    server_config = fl.server.ServerConfig(num_rounds=ENHANCED_SERVER_CONFIG["num_rounds"])

    try:
        # Start enhanced server
        result = fl.server.start_server(
            server_address="0.0.0.0:8080",
            strategy=strategy,
            config=server_config,
        )

        # Run final evaluation
        if evaluator:
            strategy.final_evaluation()

        # Display final security summary
        print(f"\n{'='*80}")
        print(f"[COMPLETED] ENHANCED SERVER SESSION COMPLETED")
        print(f"{'='*80}")
        
        security_summary = strategy.get_security_summary()
        if not security_summary.empty:
            print(f"[SUMMARY] Security Summary:")
            print(f"   Total Rounds: {len(security_summary)}")
            threat_counts = security_summary['threat_level'].value_counts()
            print(f"   Threat Distribution: {threat_counts.to_dict()}")
            avg_security = security_summary['security_score'].mean()
            print(f"   Average Security Score: {avg_security:.3f}")
            
        print(f"[FILES] Enhanced logs and reports saved to: {ENHANCED_SERVER_CONFIG['save_dir']}")
        print(f"{'='*80}")

        return result

    except Exception as e:
        logger.error(f"Enhanced server execution failed: {str(e)}")
        raise


if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser(description="Enhanced Framingham FL Server with Integrated Security")
    
    # Training arguments
    parser.add_argument("--rounds", type=int, default=10, help="Number of federated learning rounds")
    parser.add_argument("--min_clients", type=int, default=2, help="Minimum number of clients")
    parser.add_argument("--input_size", type=int, default=15, help="Number of input features")
    parser.add_argument("--save_dir", type=str, default="enhanced_checkpoints", help="Directory to save models")
    parser.add_argument("--aggregation", type=str, default="fedprox", choices=["fedavg", "fedprox"], help="Aggregation method")
    parser.add_argument("--mu", type=float, default=0.01, help="FedProx proximal term strength")
    parser.add_argument("--eval_data", type=str, default="unseandata.csv", help="Path to evaluation dataset")
    
    # Security arguments
    parser.add_argument("--security_mode", type=str, default="balanced", 
                       choices=["strict", "balanced", "permissive"], help="Security mode")
    parser.add_argument("--disable_poisoning_detection", action="store_true", help="Disable poisoning detection")
    parser.add_argument("--disable_anomaly_detection", action="store_true", help="Disable anomaly detection")
    parser.add_argument("--critical_action", type=str, default="reject_all",
                       choices=["reject_all", "use_trusted_only", "continue"], help="Action on critical threat")

    if "ipykernel" in sys.modules:
        print("Running in Jupyter/IPython environment")
        start_enhanced_server()
    else:
        args = parser.parse_args()
        
        start_enhanced_server(
            num_rounds=args.rounds,
            min_clients=args.min_clients,
            input_size=args.input_size,
            save_dir=args.save_dir,
            aggregation=args.aggregation,
            proximal_mu=args.mu,
            evaluation_data=args.eval_data,
            security_mode=args.security_mode,
            enable_detection=not args.disable_poisoning_detection,
            enable_anomaly_detection=not args.disable_anomaly_detection
        )

2025-05-24 22:45:35,404 - EnhancedFraminghamServer - WARNING - Evaluation data not found: unseandata.csv
2025-05-24 22:45:35,404 - EnhancedFraminghamServer - INFO - [SECURITY] Poisoning detection system enabled
2025-05-24 22:45:35,412 - EnhancedFraminghamServer - INFO - [SECURITY] Anomaly detection system enabled
2025-05-24 22:45:35,413 - EnhancedFraminghamServer - INFO - [SECURITY] Combined multi-layer detection enabled
2025-05-24 22:45:35,415 - EnhancedFraminghamServer - INFO - Enhanced FedProx Strategy with Multi-Layer Security initialized
	Instead, use the `flower-superlink` CLI command to start a SuperLink as shown below:

		$ flower-superlink --insecure

	To view usage and all available options, run:

		$ flower-superlink --help

	Using `start_server()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
2025-05-24 22:45:35,416 - EnhancedFraminghamServer - WARNING - DEPRECATED FEATURE: flwr.serve

Running in Jupyter/IPython environment
🔧 CONSERVATIVE POISONING DETECTOR
   Similarity threshold: 0.4
   Outlier threshold: 1.8
   Reputation threshold: 0.6
🔧 ANOMALY DETECTION SYSTEM INITIALIZED
   Statistical threshold: 2.0
   Consensus threshold: 0.2
   Logging: Enabled

[ENHANCED] FRAMINGHAM FL SERVER WITH INTEGRATED SECURITY
Training Configuration:
  Rounds:              20
  Minimum clients:     2
  Input size:          15
  Aggregation:         fedprox
  Save directory:      enhanced_model_checkpoints
  Evaluation data:     unseandata.csv
  Proximal mu:         0.01

Security Configuration:
  Security Mode:       STRICT
  Poisoning Detection: ENABLED
  Anomaly Detection:   ENABLED
  Min Trusted Clients: 1
  Max Suspicious Ratio: 0.5
  Critical Threat Action: reject_all

Detection Thresholds:
  Poisoning Similarity: 0.4
  Anomaly Statistical:  2.0
  Anomaly Consensus:    0.2
🔍 Enhanced security systems initialized and ready
📊 Comprehensive logging and monitoring enabled
🛡️ Multi-

INFO :      configure_fit: strategy sampled 2 clients (out of 2)
2025-05-24 22:45:40,083 - EnhancedFraminghamServer - INFO - configure_fit: strategy sampled 2 clients (out of 2)
INFO :      aggregate_fit: received 2 results and 0 failures
2025-05-24 22:45:41,046 - EnhancedFraminghamServer - INFO - aggregate_fit: received 2 results and 0 failures
2025-05-24 22:45:41,050 - EnhancedFraminghamServer - INFO - Round 1: No threats detected
2025-05-24 22:45:41,068 - EnhancedFraminghamServer - INFO - Round 1: No anomalies detected
2025-05-24 22:45:41,070 - EnhancedFraminghamServer - INFO - Enhanced aggregation from 2 clients with 4240 total examples
2025-05-24 22:45:41,073 - EnhancedFraminghamServer - INFO - [SAVED] Enhanced model and security data for round 1
2025-05-24 22:45:41,079 - EnhancedFraminghamServer - INFO - [SAVED] Enhanced metrics and security history
2025-05-24 22:45:41,081 - EnhancedFraminghamServer - INFO - [SECURITY] Round 1: Low threat level (Score: 0.900)
2025-05-24 22:45:41,


🔄 ENHANCED ROUND 1 AGGREGATION
Received 2 client results
📊 Extracting client data...
  Processing 1...
    Metrics: acc=0.626, loss=0.678, norm=10.378009
  Processing 5...
    Metrics: acc=0.950, loss=0.610, norm=55.457632

🔍 ENHANCED SECURITY ANALYSIS - ROUND 1
🔍 Running Poisoning Detection...

🔍 CONSERVATIVE DETECTION - ROUND 1
Analyzing 2 clients: ['1', '5']
--- High-Confidence Performance Detection ---
  High-confidence performance analysis:
    1: acc=0.626, loss=0.678, poisoning=False
    5: acc=0.950, loss=0.610, poisoning=False
--- Extreme Magnitude Detection ---
  1: magnitude = 10.378009
  5: magnitude = 55.457632
    Magnitude ratio: 5.34
--- Extreme Pattern Detection ---
  Checking for extreme patterns...
--- Conservative Validation ---
✅ REWARDED 1: 0.900 -> 0.920
✅ REWARDED 5: 0.900 -> 0.920
--- CONSERVATIVE RESULTS ---
  Suspicious: []
  Trusted: ['1', '5']
  Threat level: low
   ✅ Poisoning detection completed
🔍 Running Anomaly Detection...

🔍 ANOMALY DETECTION - ROUND

INFO :      aggregate_fit: received 2 results and 0 failures
2025-05-24 22:45:42,126 - EnhancedFraminghamServer - INFO - aggregate_fit: received 2 results and 0 failures
2025-05-24 22:45:42,130 - EnhancedFraminghamServer - INFO - Round 2: No threats detected
2025-05-24 22:45:42,148 - EnhancedFraminghamServer - INFO - Round 2: No anomalies detected
2025-05-24 22:45:42,151 - EnhancedFraminghamServer - INFO - Enhanced aggregation from 2 clients with 6360 total examples
2025-05-24 22:45:42,156 - EnhancedFraminghamServer - INFO - [SAVED] Enhanced model and security data for round 2
2025-05-24 22:45:42,160 - EnhancedFraminghamServer - INFO - [SAVED] Enhanced metrics and security history
2025-05-24 22:45:42,162 - EnhancedFraminghamServer - INFO - [SECURITY] Round 2: Low threat level (Score: 0.900)
2025-05-24 22:45:42,163 - EnhancedFraminghamServer - INFO - [PERFORMANCE] Round 2: Accuracy=0.9061, Loss=1.5033
2025-05-24 22:45:42,163 - EnhancedFraminghamServer - INFO - [AGGREGATION] Round 2: Use

...


🔄 ENHANCED ROUND 19 AGGREGATION
Received 2 client results
📊 Extracting client data...
  Processing 5...
    Metrics: acc=0.295, loss=14.944, norm=1871.915183
  Processing 1...
    Metrics: acc=0.781, loss=10.803, norm=1510.951465

🔍 ENHANCED SECURITY ANALYSIS - ROUND 19
🔍 Running Poisoning Detection...

🔍 CONSERVATIVE DETECTION - ROUND 19
Analyzing 2 clients: ['5', '1']
--- High-Confidence Performance Detection ---
  High-confidence performance analysis:
    5: acc=0.295, loss=14.944, poisoning=False
    🚨 HIGH-CONFIDENCE FLAG: 5: extremely high loss (14.944), combined poor performance (conf: 1.10)
    1: acc=0.781, loss=10.803, poisoning=False
--- Extreme Magnitude Detection ---
  5: magnitude = 1871.915183
  1: magnitude = 1510.951465
    Magnitude ratio: 1.24
--- Extreme Pattern Detection ---
  Checking for extreme patterns...
--- Conservative Validation ---
    Conservative mode: 2-client scenario
    Keeping 5: extreme confidence (1.10)
🚨 PENALIZED 5: 0.288 -> 0.193 (conf: 1.10)


INFO :      aggregate_fit: received 2 results and 0 failures
2025-05-24 22:46:00,989 - EnhancedFraminghamServer - INFO - aggregate_fit: received 2 results and 0 failures
2025-05-24 22:46:00,994 - EnhancedFraminghamServer - INFO - Round 20: No threats detected
2025-05-24 22:46:01,011 - EnhancedFraminghamServer - WARNING - Round 20: 1 anomalous clients detected: ['5']
2025-05-24 22:46:01,014 - EnhancedFraminghamServer - INFO - Enhanced aggregation from 2 clients with 5112 total examples
2025-05-24 22:46:01,019 - EnhancedFraminghamServer - INFO - [SAVED] Enhanced model and security data for round 20
2025-05-24 22:46:01,036 - EnhancedFraminghamServer - INFO - [SAVED] Enhanced metrics and security history
2025-05-24 22:46:01,039 - EnhancedFraminghamServer - INFO - [REPORT] Generated final security summary
2025-05-24 22:46:01,042 - EnhancedFraminghamServer - WARNING - [SECURITY] Round 20: Medium threat level
2025-05-24 22:46:01,043 - EnhancedFraminghamServer - WARNING -   Security Score: 0.4


🔄 ENHANCED ROUND 20 AGGREGATION
Received 2 client results
📊 Extracting client data...
  Processing 5...
    Metrics: acc=0.950, loss=17.535, norm=1724.511312
  Processing 1...
    Metrics: acc=0.773, loss=11.153, norm=1510.971284

🔍 ENHANCED SECURITY ANALYSIS - ROUND 20
🔍 Running Poisoning Detection...

🔍 CONSERVATIVE DETECTION - ROUND 20
Analyzing 2 clients: ['5', '1']
--- High-Confidence Performance Detection ---
  High-confidence performance analysis:
    5: acc=0.950, loss=17.535, poisoning=False
    1: acc=0.773, loss=11.153, poisoning=False
--- Extreme Magnitude Detection ---
  5: magnitude = 1724.511312
  1: magnitude = 1510.971284
    Magnitude ratio: 1.14
--- Extreme Pattern Detection ---
  Checking for extreme patterns...
--- Conservative Validation ---
✅ REWARDED 5: 0.174 -> 0.194
✅ REWARDED 1: 0.277 -> 0.297
--- CONSERVATIVE RESULTS ---
  Suspicious: []
  Trusted: []
  Threat level: low
   ✅ Poisoning detection completed
🔍 Running Anomaly Detection...

🔍 ANOMALY DETECTION 

2025-05-24 22:46:01,197 - EnhancedFraminghamServer - INFO - 	History (loss, distributed):
INFO :      		round 1: 1.5043562352657318
2025-05-24 22:46:01,197 - EnhancedFraminghamServer - INFO - 		round 1: 1.5043562352657318
INFO :      		round 2: 1.513416975736618
2025-05-24 22:46:01,203 - EnhancedFraminghamServer - INFO - 		round 2: 1.513416975736618
INFO :      		round 3: 1.439226120710373
2025-05-24 22:46:01,205 - EnhancedFraminghamServer - INFO - 		round 3: 1.439226120710373
INFO :      		round 4: 0.5256025791168213
2025-05-24 22:46:01,205 - EnhancedFraminghamServer - INFO - 		round 4: 0.5256025791168213
INFO :      		round 5: 1.1832747161388397
2025-05-24 22:46:01,209 - EnhancedFraminghamServer - INFO - 		round 5: 1.1832747161388397
INFO :      		round 6: 1.4660179316997528
2025-05-24 22:46:01,212 - EnhancedFraminghamServer - INFO - 		round 6: 1.4660179316997528
INFO :      		round 7: 4.699153780937195
2025-05-24 22:46:01,220 - EnhancedFraminghamServer - INFO - 		round 7: 4.69915378


[COMPLETED] ENHANCED SERVER SESSION COMPLETED
[SUMMARY] Security Summary:
   Total Rounds: 19
   Threat Distribution: {'low': 12, 'medium': 7}
   Average Security Score: 0.734
[FILES] Enhanced logs and reports saved to: enhanced_model_checkpoints
